In [1]:
# after running 2-build_neuralnets, you will obtain the best configuration of an NN model
# rerun the model with "identified best configuration"
# then save the model for future use

In [2]:
#
# for laziness, i copied paste the code in 2-xxx, you can save the data in 2-xxx, to avoid the overhead
#

In [3]:
import os,sys
from sklearn import preprocessing

import json
import codecs
import yaml
import numpy as np
import pandas as pd

sys.path.append('../utils')
from cudaMetrics import *

In [4]:
df_app_scale = pd.read_csv('app_proflingmetrics_scaled.csv')
df_metrics = df_app_scale.drop(df_app_scale.columns[0], axis=1) # drop the 1st column

fastdev_dd = np.load("../05_whichgpufast/home-fastdev.npy").item()
targetpps = list(fastdev_dd.keys())

df_metrics_cp = df_metrics.copy()
for index, row in df_metrics_cp.iterrows():
    appName = row['AppName']
    if not appName in targetpps:
        print("[warning] delete current row in the dataframe ({})".format(appName))
        df_metrics.drop(df_metrics[df_metrics.AppName == appName].index, inplace=True)


df_metrics['best_dev'] = 1 
for index, row in df_metrics.iterrows():
    appName =  row['AppName']
    fastdev = fastdev_dd[appName]
    df_metrics.loc[index, 'best_dev'] = fastdev
    

def gen_model_input(df_dataset):
    df_X = df_dataset.drop(['AppName', 'best_dev'], axis=1)
    df_y = df_dataset['best_dev']
    df_y = df_y.astype('int64')  # convert obj to int
    return df_X, df_y

df_X, df_y = gen_model_input(df_metrics)
df_X = df_X.as_matrix().astype(np.float)
df_y = df_y.as_matrix().astype(np.int32)

[warning] delete current row in the dataframe (rodinia_heartwall)


/Users/leiming/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:29: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/leiming/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:30: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [6]:
from sklearn.model_selection import StratifiedKFold  # StratifiedKFold
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
import pickle

### build NN model with the target param

In [ ]:
pkl_filename = "output_model.pkl"

fold_k = 5
kf = StratifiedKFold(n_splits=fold_k, random_state=314159, shuffle=True)

minError = 1.0
for train_index, test_index in kf.split(df_X, df_y):            
    X_train, X_test = df_X[train_index], df_X[test_index]
    y_train, y_test = df_y[train_index], df_y[test_index]

    clsfy = MLPClassifier(hidden_layer_sizes=(100, 100, 100),
                          activation='identity',
                          solver='adam',
                          alpha=1.0,
                          max_iter=1000) # max 1K iterations
            
    clsfy.fit(X_train, y_train)
    err = metrics.mean_absolute_error(y_test, clsfy.predict(X_test))
    print("error : {}", err)
    if err < minError:
        # save the model 
        with open(pkl_filename, 'wb') as file: pickle.dump(model, file)